In [ ]:
#考可能虑到拼接图像宽高不一致, 或拍摄角度，垂直移动问题
import cv2
import numpy as np

def stitch_two_images(img1, img2):
    # 将彩色图像转换为灰度图
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # 创建SIFT检测器
    sift = cv2.SIFT_create()

    # 检测关键点和描述符
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    # 匹配描述符
    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(des1, des2, k=2)

    # 应用比率测试
    good_matches = []
    for m,n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # 计算变换矩阵
    if len(good_matches) > 4:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        matrix, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # 计算输出图像的尺寸
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        points1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
        points2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
        points2_ = cv2.perspectiveTransform(points1, matrix)
        points = np.concatenate((points2, points2_), axis=0)
        [x_min, y_min] = np.int32(points.min(axis=0).ravel() - 0.5)
        [x_max, y_max] = np.int32(points.max(axis=0).ravel() + 0.5)
        translation_dist = [-x_min, -y_min]
        H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])

        # 用变换矩阵变换第一张图
        output_img = cv2.warpPerspective(img1, H_translation.dot(matrix), (x_max-x_min, y_max-y_min))
        output_img[translation_dist[1]:translation_dist[1]+h2, translation_dist[0]:translation_dist[0]+w2] = img2

        return output_img
    else:
        return None

#将图像列表中图像逐一拼接
def q_stitch_images(images):
    current_part = images[0]
    for i in range(1, len(images)):
        current_part = stitch_two_images(current_part, images[i])
        if current_part is None:
            raise Exception("Error in stitching images at index {}".format(i))
    return current_part


def main():
# 加载图像  可修改为实际拼接图像路径
    # img1 = cv2.imread('/home/kh/homework/result/test1_1.jpg')
    # img2 = cv2.imread('/home/kh/homework/result/test1_2.jpg')
    # img3 = cv2.imread('/home/kh/homework/result/test1_3.jpg')
    # images = [img1,img2,img3]
    img1 = cv2.imread('/home/kh/homework/result/a1.jpg')
    img2 = cv2.imread('/home/kh/homework/result/a2.jpg')
    images = [img1,img2]
# 图像拼接
    result = q_stitch_images(images)
    if result is not None:
        image_name = 'a' + ".jpg" #图像命名：.jpg    
        cv2.imwrite( '/home/kh/homework/result/' + image_name, result)  #保存
        print("图像拼接成功！")
    else:
        print("图像拼接失败！") 


if __name__ =="__main__":     
    main()